In [1]:
import numpy as np
import pandas as pd
from pyproj import Proj, transform

accidents = pd.read_pickle('../fars_data/final_accidents2.pickle')
drivers = pd.read_pickle('../fars_data/final_drivers.pickle')

In [2]:
from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")

def convert(longitudes, latitudes):
    """Converts latlon coordinates to meters.
    Inputs:
    longitudes (array-like) : array of longitudes
    latitudes (array-like) : array of latitudes
    Example:
    x,y = convert(accidents.LONGITUD, accidents.LATITUDE)
    """
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    return x_vals, y_vals

accidents["x"], accidents["y"] = convert(accidents.LONGITUD, accidents.LATITUDE)

In [3]:
from bokeh.plotting import Figure
from bokeh.models import WMTSTileSource

fig = Figure(plot_width=1100, plot_height=650,
            x_range=(-13000000, -7000000), y_range=(2750000, 6250000),
            tools=["wheel_zoom", "pan"], active_scroll="wheel_zoom", webgl=True)

fig.axis.visible = False

STAMEN_TONER_BACKGROUND = WMTSTileSource(
    url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png',
    attribution=(
        'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
        'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
        'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
        'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'
    )
)

fig.add_tile(STAMEN_TONER_BACKGROUND)

TileRenderer(id='888ada4d-57af-4e7d-bd42-6765ebd169cc', ...)

In [5]:
# Replicate the circle marker plot using a DataFrame and ColumnDataSource.
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

df_drunk_speeding = accidents[(accidents['DRUNK_DR'] > 0)&(accidents['SP'] > 0)]
df_drunk = accidents[(accidents['DRUNK_DR'] > 0)&(accidents['SP'] == 0)]
df_speeding = accidents[(accidents['DRUNK_DR'] == 0)&(accidents['SP'] > 0)]
df_other = accidents[(accidents['DRUNK_DR'] == 0)&(accidents['SP'] == 0)]

cir2 = fig.circle(x="x", y="y", source=df_drunk,
                size=0.5, color="red")

cir3 = fig.circle(x="x", y="y", source=df_speeding,
                size=0.5, color="blue")

cir1 = fig.circle(x="x", y="y", source=df_drunk_speeding,
                size=0.5, color="purple")

cir4 = fig.circle(x="x", y="y", source=df_other,
                size=0.5, color= "black")

show(fig)